In [1]:
from models.models import *
import torch



$\frac{\partial u_i}{\partial t} = - \frac{d p}{\rho d x_i} - u_j \frac{\partial u_i}{\partial x_j} + \frac{1}{Re} \frac{\partial^2 u_j}{\partial x_i \partial x_j}$


$\frac{du_i}{dx_i} = 0$

In [ ]:
a = torch.tensor([2., 3.], requires_grad=True)
b = torch.tensor([6., 4.], requires_grad=True)

Q = 3*a**3 - b**2
external_grad = torch.tensor([1., 1.])


test = torch.arange(10,dtype=torch.float, requires_grad=True)
external_grad = torch.tensor([0.5]*10)

grad = test.backward(gradient=external_grad)
print(test.grad)

# simple biquadratic approximation for unstructured derivitive calculation
# value = a0+a1x+a2x^2+a3y+a4y^2+a5xy

# construct matrix
coeff_mat = torch.empty(6*nodes*2,6*nodes*2)
coeff_mat[:,0] = 0

coeff_mat[:,1] = e[]

loss = dudx+dvdy
loss.backward()
# update to p


tensor([0.5000, 0.5000, 0.5000, 0.5000, 0.5000, 0.5000, 0.5000, 0.5000, 0.5000,
        0.5000])


In [ ]:
class FD(torch.nn.Module):
    def __init__(self, *args, **kwargs) -> None:
        super().__init__(*args, **kwargs)
        pass

    def forward(self, v: torch.Tensor, e: torch.Tensor, ij: torch.Tensor) -> torch.Tensor:
        # v stores [u,v]
        # ij stores connections
        # e stores [x_rel,y_rel]

        


In [ ]:

class FD(torch.nn.Module):
    def __init__(self, dt: float = 0.01, rho: float = 1.0, nu: float = 0.01, relaxation_factor: float = 0.2) -> None:
        super().__init__()
        self.dt = dt
        self.nu = nu
        self.rho = rho
        self.relaxation_factor = relaxation_factor

    def forward(self, v: torch.Tensor, e: torch.Tensor, ij: torch.Tensor) -> torch.Tensor:
        """
        v: [N, 2] -> Velocity (u, v) at N nodes.
        e: [M, 2] -> Relative distances (dx, dy) for M edges.
        ij: [2, M] -> Connectivity (source i, neighbor j).
        """
        num_nodes = v.size(0)
        idx_i, idx_j = ij[0], ij[1]

        # 1. Compute Differences (Message Passing)
        # dv[m] = v[j] - v[i] for edge m
        dv = v[idx_j] - v[idx_i]  # [M, 2]

        # 2. Gradient Reconstruction (Least Squares / Weighted Avg)
        # We approximate (du/dx, du/dy) and (dv/dx, dv/dy) at each edge
        dist_sq = torch.sum(e**2, dim=1, keepdim=True) + 1e-8
        
        # Local directional derivatives per edge
        # grad_u[m] is (du/dx, du/dy) approximated from edge m
        grad_u_edges = (dv[:, 0] * e) / dist_sq
        grad_v_edges = (dv[:, 1] * e) / dist_sq

        # Aggregate edge-wise gradients back to nodes (Mean of neighbors)
        node_grad_u = torch.zeros((num_nodes, 2), device=v.device).index_add_(0, idx_i, grad_u_edges)
        node_grad_v = torch.zeros((num_nodes, 2), device=v.device).index_add_(0, idx_i, grad_v_edges)

        # 3. Convective Term: (u · ∇)u
        # For each node: u * du/dx + v * du/dy
        conv_u = v[:, 0] * node_grad_u[:, 0] + v[:, 1] * node_grad_u[:, 1]
        conv_v = v[:, 0] * node_grad_v[:, 0] + v[:, 1] * node_grad_v[:, 1]
        convection = torch.stack([conv_u, conv_v], dim=1)

        # 4. Viscous Term: ν ∇²u
        # Laplacian approximated via the second-order finite difference on graphs
        # L(u) ≈ Σ (u_j - u_i) / ||e_ij||^2
        laplacian = torch.zeros_like(v)
        laplacian.index_add_(0, idx_i, dv / dist_sq)


        est_p = torch.ones(num_nodes)
        while True:
            dp = est_p[idx_j] - est_p[idx_i]

            #  Pressure corrections: dp/dx/rho
            grad_p_edges = (dp * e) / dist_sq

            # Aggregate edge-wise gradients back to nodes (Mean of neighbors)
            node_grad_p = torch.zeros((num_nodes, 2), device=v.device).index_add_(0, idx_i, grad_p_edges)

            pressure_force = node_grad_p / self.rho

            # 5. Predictor Step: Forward Euler
            # v* = v + dt * ( - (u·∇)u + ν ∇²u )
            dv_dt = -convection + self.nu * laplacian - pressure_force
            v_next = v + self.dt * dv_dt

            # v_next.backward()

            dv_est = v_next[idx_j] - v_next[idx_i]  # [M, 2]
            
            # Local directional derivatives per edge
            # grad_u[m] is (du/dx, du/dy) approximated from edge m
            grad_u_est_edges = (dv_est[:, 0] * e) / dist_sq
            grad_v_est_edges = (dv_est[:, 1] * e) / dist_sq

            # Aggregate edge-wise gradients back to nodes (Mean of neighbors)
            node_grad_u_est = torch.zeros((num_nodes, 2), device=v.device).index_add_(0, idx_i, grad_u_est_edges)
            node_grad_v_est = torch.zeros((num_nodes, 2), device=v.device).index_add_(0, idx_i, grad_v_est_edges)

            divergence = node_grad_u_est[:,0] + node_grad_v_est[:,1]
            if torch.all(divergence<1e-4):
                break
            divergence.backward(est_p)
            est_p += divergence.grad*self.relaxation_factor


        return v_next

In [24]:
import torch
import torch.nn as nn

class FD(nn.Module):
    def __init__(
        self,
        dt: float = 0.01,
        rho: float = 1.0,
        nu: float = 0.01,
        relaxation_factor: float = 0.2,
        poisson_iters: int = 50,
    ):
        super().__init__()
        self.dt = dt
        self.nu = nu
        self.rho = rho
        self.relaxation_factor = relaxation_factor
        self.poisson_iters = poisson_iters

    # -------------------------------------------------------
    # Differential operators
    # -------------------------------------------------------

    def gradient(self, p, e, ij):
        """
        ∇p at nodes from edge differences
        """
        i, j = ij
        dp = p[j] - p[i]                     # [M]
        grad_e = dp[:, None] * e             # [M, 2]

        grad = torch.zeros_like(p[:, None].repeat(1, 2))
        grad.index_add_(0, i, grad_e)
        grad.index_add_(0, j, -grad_e)

        return grad

    def divergence(self, v, e, ij):
        """
        ∇·v at nodes
        """
        i, j = ij
        dv = v[j] - v[i]                     # [M, 2]
        flux = (dv * e).sum(dim=1)           # [M]

        div = torch.zeros(v.shape[0], device=v.device)
        div.index_add_(0, i, flux)
        div.index_add_(0, j, -flux)

        return div

    def laplacian(self, v, e, ij):
        """
        Vector Laplacian using edge diffusion
        """
        i, j = ij
        dv = v[j] - v[i]                     # [M, 2]
        w = (e ** 2).sum(dim=1, keepdim=True)  # |e|^2

        diff = dv / (w + 1e-8)

        lap = torch.zeros_like(v)
        lap.index_add_(0, i, diff)
        lap.index_add_(0, j, -diff)

        return lap

    # -------------------------------------------------------
    # Pressure Poisson solver (Jacobi)
    # -------------------------------------------------------

    def solve_pressure(self, rhs, e, ij):
        p = torch.zeros_like(rhs)

        i, j = ij
        w = (e ** 2).sum(dim=1)
        w = torch.clamp(w, min=1e-4)

        # Precompute diagonal
        diag = torch.zeros_like(p)
        diag.index_add_(0, i, 1.0 / w)
        diag.index_add_(0, j, 1.0 / w)
        diag = torch.clamp(diag, min=1e-6)

        for _ in range(self.poisson_iters):
            dp = p[j] - p[i]
            flux = dp / w

            lap = torch.zeros_like(p)
            lap.index_add_(0, i, flux)
            lap.index_add_(0, j, -flux)

            # Jacobi update
            p = p + self.relaxation_factor * (rhs - lap) / diag

            # Nullspace removal
            p = p - p.mean()

        return p


    # -------------------------------------------------------
    # Main time step
    # -------------------------------------------------------

    def forward(self, v, e, ij):
        """
        v: [N, 2] velocity
        e: [M, 2] edge vectors
        ij: [2, M] connectivity
        """

        # ---------------------------
        # Convection (edge upwind-ish)
        # ---------------------------
        i, j = ij
        v_avg = 0.5 * (v[i] + v[j])
        dv = v[j] - v[i]
        conv_flux = (v_avg * e).sum(dim=1, keepdim=True) * dv

        conv = torch.zeros_like(v)
        conv.index_add_(0, i, conv_flux)
        conv.index_add_(0, j, -conv_flux)

        # ---------------------------
        # Diffusion
        # ---------------------------
        diff = self.nu * self.laplacian(v, e, ij)

        # ---------------------------
        # Predictor step
        # ---------------------------
        v_star = v + self.dt * (-conv + diff)

        # ---------------------------
        # Pressure projection
        # ---------------------------
        div_v = self.divergence(v_star, e, ij)
        rhs = (self.rho / self.dt) * div_v

        p = self.solve_pressure(rhs, e, ij)

        grad_p = self.gradient(p, e, ij)
        v_new = v_star - (self.dt / self.rho) * grad_p

        return v_new, p


In [25]:
import torch
import numpy as np

def make_unstructured_graph(n=200, radius=0.2):
    """
    Generates:
      nodes in [0,1]²
      edges based on radius search
    """
    pts = torch.rand(n, 2)

    edges_i = []
    edges_j = []
    edges_e = []

    for i in range(n):
        diff = pts - pts[i]
        dist = torch.norm(diff, dim=1)

        nbrs = (dist < radius) & (dist > 1e-6)
        for j in torch.where(nbrs)[0]:
            edges_i.append(i)
            edges_j.append(j.item())
            edges_e.append(diff[j])

    ij = torch.tensor([edges_i, edges_j], dtype=torch.long)
    e = torch.stack(edges_e)

    return pts, e, ij

def initialize_velocity(x):
    """
    Random velocity with bias
    """
    v = torch.randn_like(x)
    v[:, 0] += 1.0   # induce bulk flow
    return v

def divergence_norm(model, v, e, ij):
    div = model.divergence(v, e, ij)
    return torch.norm(div) / v.shape[0]

# from fd_model import FD   # or wherever your FD class lives

torch.manual_seed(0)

# Graph
x, e, ij = make_unstructured_graph(n=300, radius=0.15)

# Model
model = FD(
    dt=0.01,
    nu=0.01,
    rho=1.0,
    poisson_iters=40,
    relaxation_factor=0.25,
)

# Velocity
v0 = initialize_velocity(x)
v0.requires_grad_(True)

# Before projection
div0 = divergence_norm(model, v0, e, ij)

# Step
v1, p = model(v0, e, ij)

# After projection
div1 = divergence_norm(model, v1, e, ij)

print(f"Divergence before: {div0.item():.3e}")
print(f"Divergence after : {div1.item():.3e}")
print(f"Reduction factor : {(div1/div0).item():.3f}")


Divergence before: 5.268e-09
Divergence after : 1.211e-08
Reduction factor : 2.299


In [26]:
v = v0.detach()
for step in range(20):
    v, p = model(v, e, ij)

    if step % 5 == 0:
        div = divergence_norm(model, v, e, ij)
        ke = torch.mean(torch.sum(v**2, dim=1))
        print(f"step {step:02d}: div={div:.3e}, KE={ke:.3e}")


step 00: div=1.211e-08, KE=2.073e+01
step 05: div=7.872e-01, KE=1.116e+17
step 10: div=nan, KE=nan
step 15: div=nan, KE=nan
